# Pandas

Pandas is the main library for reading and manipulating data. It is built on top of NumPy and provides an easy-to-use data structure for data analysis. Its used in fields such as machine learning, data science, and data analysis.

In [ ]:
# Online version only or if you dont have it
%pip install pandas
# Conda works too
# %conda install pandas

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
import pandas as pd
pd.__version__

The primary data structures in *pandas* are implemented as two classes:

  * **`DataFrame`**, which you can imagine as a relational data table, with rows and named columns.
  * **`Series`**, which is a single column. A `DataFrame` contains one or more `Series` and a name for each `Series`.

One way to create a `Series` is to construct a `Series` object. For example:

In [ ]:
pd.Series(['San Francisco', 'San Jose', 'Sacramento'])

`DataFrame` objects can be created by passing a `dict` mapping `string` column names to their respective `Series`. If the `Series` don't match in length, missing values are filled with special [NA/NaN](http://pandas.pydata.org/pandas-docs/stable/missing_data.html) values. Example:

In [ ]:
city_names = pd.Series(['San Francisco', 'San Jose', 'Sacramento'])
population = pd.Series([852469, 1015785, 485199])

pd.DataFrame({ 'City name': city_names, 'Population': population })

But most of the time, you load an entire file into a `DataFrame`. The following example loads a file with California housing data. Run the following cell to load the data and create feature definitions:

In [ ]:
california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")
california_housing_dataframe

In [ ]:
california_housing_dataframe.describe()

In [ ]:
'longitude' in california_housing_dataframe.columns

In [ ]:
california_housing_dataframe.head(10)

In [ ]:
california_housing_dataframe.tail(10)

In [ ]:
california_housing_dataframe.hist('housing_median_age')

You can also read excel files using read_excel. 

Note: You need to install ``openpyxl`` for this to work

In [ ]:
%pip install openpyxl

In [ ]:

df_cuvette = pd.read_excel(
    "../data/Cuvette.xlsx",
    skiprows=2,
)

df_cuvette

In [ ]:
pd.read_excel?

## Accessing Data

You can access `DataFrame` data like a dict/list

In [ ]:
cities = pd.DataFrame({ 'City name': city_names, 'Population': population })
cities

In [ ]:
cities['Population']

In [ ]:
cities[1:3]

In [ ]:
bedrooms = california_housing_dataframe['total_bedrooms']
bedrooms

## Manipulating Data

You may apply Python's basic arithmetic operations to `Series`. For example:

In [ ]:
population / 1000.0

And many numpy operations also work on it too!

In [ ]:
import numpy as np
np.log10(population)

In [ ]:
np.sum(bedrooms)

We can also apply more complicated operations using the `apply` method by passing in a function!

Why not try a lambda function?

In [ ]:
population.apply(lambda x: x > 500_000)

Modifying `DataFrames` is also straightforward. For example, the following code adds two `Series` to an existing `DataFrame`:

In [ ]:
cities

In [ ]:
cities['Area square miles'] = pd.Series([46.87, 176.53, 97.92])
cities['Population density'] = cities['Population'] / cities['Area square miles']
cities

## Exercise #1

Modify the `cities` table by adding a new boolean column called `Wide and saint` that is True if and only if *both* of the following are True:

  * The city is named after a saint.
  * The city has an area greater than 50 square miles.

**Note:** The filtering rules previously discussed in the numpy notebook may be useful.

**Note:** strings have a function called `startswith`

**Hint:** "San" in Spanish means "saint."

In [ ]:
saint_cities = cities['City name'].apply(lambda x: 'San' in x)
square_miles_fifty = cities['Area square miles'] > 50.0
cities['Wide and saint'] = square_miles_fifty & saint_cities
cities

In [ ]:
if any((cities['Area square miles'] > 50.0) & (cities['City name'].apply(lambda x: 'San' in x))):
    print('Yay')

In [ ]:
bedrooms.values

In [ ]:
plt.figure()
plt.plot(bedrooms)
plt.show()

## Sorting and indexing

We can sort our california dataframe using `sort_values`

In [ ]:
new_df = california_housing_dataframe.sort_values('median_house_value',inplace=False, ascending=True, ignore_index=True)
new_df

In [ ]:
california_housing_dataframe.sort_values?

#### Difference between loc and iloc

- `.loc` selection is based on the value of the index. For example if the index was categorical we could index via some category. 
- `.iloc` selection is **always** based on integer positions. When using iloc we are treating the dataframe as 2d-array with no special structure compared to the case of `.loc`

In [ ]:
california_housing_dataframe.iloc[0]

In [ ]:
california_housing_dataframe.loc[568]

In [ ]:
cities.loc[0]

Slicing is valid with `iloc`

In [ ]:
california_housing_dataframe.iloc[:-50:-1]

In [ ]:
#Code here

Lets load a different dataset now:

In [ ]:
titanic = pd.read_csv('../data/titanic.csv')
titanic

### Filtering Dataframes

You can filter data based on the columns and values in the dataframe exactly like numpy:

In [ ]:
titanic[titanic.Sex == 'male']

You can also filter for a specific `Series` as well

In [ ]:
titanic.Age[titanic.Sex=='male']

Exactly like numpy you can apply the same filtering rules using the same operators (`&`, `|`, `^`, `~`)

In [ ]:
titanic.Survived[ (titanic.Sex=='male') & (titanic.Age>=18) ].mean()

In [ ]:
titanic.Survived[(titanic.Sex=='female')&(titanic.Age>=18)].mean()

In [ ]:
titanic.Survived[titanic.Pclass == 1].mean()

## Groupby

Groupby allows you to collect the same classes together within a column for example. To perform what we did on all classes of sex as before we can istead do:

In [ ]:
titanic.groupby('Sex')['Survived'].mean()

In [ ]:
titanic.groupby('Embarked')['Survived'].mean()

You can even group by multiple classes

In [ ]:
titanic.groupby(['Pclass', 'Sex'])['Survived'].mean()

In [ ]:
titanic[titanic.Age < 18].groupby('Sex')['Survived'].mean()

And if we add another column we can create an entirely new dataframe:

In [ ]:
new = titanic.groupby(['Sex','Pclass'])[['Survived','Age']].mean()
new

## Exercises

#### What was the average age of the survivors?

In [ ]:
titanic.Age[titanic.Survived == 1].mean()

#### What was the combined survival rate of both children (age less than 18) and seniors (age greater than 60)?

In [ ]:
# Child rate here
children_rate = titanic.Survived[(titanic.Age < 18)].mean()
children_rate

In [ ]:
# Senior rate here
senior_rate = titanic.Survived[(titanic.Age > 60)].mean()
senior_rate

In [ ]:
# Combined rate here
print(f'{children_rate + senior_rate:.2%}')

#### Group by pClass and investigate average survival rate, age and fare


In [ ]:
titanic.groupby('Pclass')[['Survived', 'Age', 'Fare']].mean()